In [39]:
import argparse
import pickle
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import math, copy
from torch import Tensor
from matplotlib import pyplot as plt
import torch.optim as optim
import random
import tqdm
from torch import autograd
import time
from models import sahp

## Power Hawkes - SAHP

In [40]:
chosen_file ='../data/simulated/power_hawkes/power_hawkes.pkl'
with open(chosen_file, 'rb') as f:
    loaded_hawkes_data = pickle.load(f)
tmax =loaded_hawkes_data['tmax']
process_dim = loaded_hawkes_data['process_dim'] if 'process_dim' in loaded_hawkes_data.keys() else process_dim
seq_times, seq_types, seq_lengths, _,seq_intensities = sahp.process_loaded_sequences(loaded_hawkes_data, process_dim)

total_sample_size = seq_times.size(0)
print("Total sample size: {}".format(total_sample_size))

train_ratio = 0.8
train_size = int(train_ratio * total_sample_size)
dev_ratio = 0.1
dev_size = int(dev_ratio * total_sample_size)
test_size = int(dev_ratio * total_sample_size)

print("Train sample size: {:}/{:}".format(train_size, total_sample_size))
print("Dev sample size: {:}/{:}".format(dev_size, total_sample_size))


# Define training data
train_times_tensor = seq_times[:train_size]
train_seq_types = seq_types[:train_size]
train_seq_lengths = seq_lengths[:train_size]
print("No. of event tokens in training subset:", train_seq_lengths.sum())

# Define development data
dev_times_tensor = seq_times[train_size:train_size + dev_size]  # train_size+dev_size
dev_seq_types = seq_types[train_size:train_size + dev_size]
dev_seq_lengths = seq_lengths[train_size:train_size + dev_size]
print("No. of event tokens in development subset:", dev_seq_lengths.sum())

test_times_tensor = seq_times[-dev_size:]
test_seq_types = seq_types[-dev_size:]
test_seq_lengths = seq_lengths[-dev_size:]
# test_seq_intensities = seq_intensities[-dev_size:]

print("No. of event tokens in test subset:", test_seq_lengths.sum())


train_seq_lengths, reorder_indices_train = train_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
train_times_tensor = train_times_tensor[reorder_indices_train]
train_seq_types = train_seq_types[reorder_indices_train]
#
dev_seq_lengths, reorder_indices_dev = dev_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
dev_times_tensor = dev_times_tensor[reorder_indices_dev]
dev_seq_types = dev_seq_types[reorder_indices_dev]

test_seq_lengths, reorder_indices_test = test_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
test_times_tensor = test_times_tensor[reorder_indices_test]
test_seq_types = test_seq_types[reorder_indices_test]
# test_seq_intensities = test_seq_intensities[reorder_indices_test]

device = 'cpu'
max_sequence_length = max(train_seq_lengths[0], dev_seq_lengths[0], test_seq_lengths[0])
print('max_sequence_length: {}'.format(max_sequence_length))
model = sahp.make_model(nLayers=4, d_model=16, atten_heads=8,
                   dropout=0.1, process_dim=1, device=device, pe='add',
                   max_sequence_length=max_sequence_length + 1).to(device)

Total sample size: 4000
Train sample size: 3200/4000
Dev sample size: 400/4000
No. of event tokens in training subset: tensor(169435)
No. of event tokens in development subset: tensor(21338)
No. of event tokens in test subset: tensor(21076)
max_sequence_length: 188


In [41]:
model_dict =torch.load('trained_models/sahp/power_hawkes_debug_model.pt',map_location=torch.device('cpu'))
model.load_state_dict(model_dict)

<All keys matched successfully>

In [42]:
test_rmse, test_accuracy, results = \
    sahp.prediction_evaluation(device, model, test_seq_lengths, test_times_tensor, test_seq_types, test_size, tmax)

test_rmse, test_accuracy, results = \
    sahp.prediction_evaluation(device, model, dev_seq_lengths, dev_times_tensor, dev_seq_types, test_size, tmax)


test_rmse, test_accuracy, results = \
    sahp.prediction_evaluation(device, model, train_seq_lengths, train_times_tensor, train_seq_types, train_size, tmax)

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:05<00:00, 78.76it/s]


rmse 7.509196936773859
Type prediction score: 1.0


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:04<00:00, 89.52it/s]


rmse 7.635167831245053
Type prediction score: 1.0


100%|██████████████████████████████████████████████████████████████████████████████| 3200/3200 [00:33<00:00, 94.99it/s]


rmse 7.746266111609838
Type prediction score: 1.0


## Sin Hawkes

In [45]:
chosen_file ='../data/simulated/sin_hawkes/sinusodial_1d_hawkes.pkl'
with open(chosen_file, 'rb') as f:
    loaded_hawkes_data = pickle.load(f)
tmax =loaded_hawkes_data['tmax']
process_dim = loaded_hawkes_data['process_dim'] if 'process_dim' in loaded_hawkes_data.keys() else process_dim
seq_times, seq_types, seq_lengths, _,seq_intensities = sahp.process_loaded_sequences(loaded_hawkes_data, process_dim)

total_sample_size = seq_times.size(0)
print("Total sample size: {}".format(total_sample_size))

train_ratio = 0.8
train_size = int(train_ratio * total_sample_size)
dev_ratio = 0.1
dev_size = int(dev_ratio * total_sample_size)
test_size = int(dev_ratio * total_sample_size)

print("Train sample size: {:}/{:}".format(train_size, total_sample_size))
print("Dev sample size: {:}/{:}".format(dev_size, total_sample_size))


# Define training data
train_times_tensor = seq_times[:train_size]
train_seq_types = seq_types[:train_size]
train_seq_lengths = seq_lengths[:train_size]
print("No. of event tokens in training subset:", train_seq_lengths.sum())

# Define development data
dev_times_tensor = seq_times[train_size:train_size + dev_size]  # train_size+dev_size
dev_seq_types = seq_types[train_size:train_size + dev_size]
dev_seq_lengths = seq_lengths[train_size:train_size + dev_size]
print("No. of event tokens in development subset:", dev_seq_lengths.sum())

test_times_tensor = seq_times[-dev_size:]
test_seq_types = seq_types[-dev_size:]
test_seq_lengths = seq_lengths[-dev_size:]
# test_seq_intensities = seq_intensities[-dev_size:]

print("No. of event tokens in test subset:", test_seq_lengths.sum())


train_seq_lengths, reorder_indices_train = train_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
train_times_tensor = train_times_tensor[reorder_indices_train]
train_seq_types = train_seq_types[reorder_indices_train]
#
dev_seq_lengths, reorder_indices_dev = dev_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
dev_times_tensor = dev_times_tensor[reorder_indices_dev]
dev_seq_types = dev_seq_types[reorder_indices_dev]

test_seq_lengths, reorder_indices_test = test_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
test_times_tensor = test_times_tensor[reorder_indices_test]
test_seq_types = test_seq_types[reorder_indices_test]
# test_seq_intensities = test_seq_intensities[reorder_indices_test]

device = 'cpu'
max_sequence_length = max(train_seq_lengths[0], dev_seq_lengths[0], test_seq_lengths[0])
print('max_sequence_length: {}'.format(max_sequence_length))
model = sahp.make_model(nLayers=4, d_model=16, atten_heads=8,
                   dropout=0.1, process_dim=1, device=device, pe='add',
                   max_sequence_length=max_sequence_length + 1).to(device)


model_dict =torch.load('trained_models/sahp/sinusodial_1d_hawkes_debug_model.pt',map_location=torch.device('cpu'))
model.load_state_dict(model_dict)

Total sample size: 4000
Train sample size: 3200/4000
Dev sample size: 400/4000
No. of event tokens in training subset: tensor(170178)
No. of event tokens in development subset: tensor(21036)
No. of event tokens in test subset: tensor(21093)
max_sequence_length: 91


<All keys matched successfully>

In [44]:
test_rmse, test_accuracy, results = \
    sahp.prediction_evaluation(device, model, test_seq_lengths, test_times_tensor, test_seq_types, test_size, tmax)

test_rmse, test_accuracy, results = \
    sahp.prediction_evaluation(device, model, dev_seq_lengths, dev_times_tensor, dev_seq_types, test_size, tmax)


test_rmse, test_accuracy, results = \
    sahp.prediction_evaluation(device, model, train_seq_lengths, train_times_tensor, train_seq_types, train_size, tmax)

100%|███████████████████████████████████████████████████████████████████████████████| 400/400 [00:02<00:00, 156.35it/s]


rmse 5.588189456739897
Type prediction score: 1.0


100%|███████████████████████████████████████████████████████████████████████████████| 400/400 [00:02<00:00, 141.50it/s]


rmse 5.996967503011646
Type prediction score: 1.0


100%|█████████████████████████████████████████████████████████████████████████████| 3200/3200 [00:27<00:00, 117.43it/s]


rmse 5.803682263332539
Type prediction score: 1.0


## 2-D Hawkes

In [4]:
torch.cuda.manual_seed(seed=42)
torch.cuda.manual_seed_all(42)

In [5]:
chosen_file ='../data/simulated/2_d_hawkes/hawkes_2d.pkl'
with open(chosen_file, 'rb') as f:
    loaded_hawkes_data = pickle.load(f)
tmax =loaded_hawkes_data['tmax']
process_dim = loaded_hawkes_data['process_dim'] if 'process_dim' in loaded_hawkes_data.keys() else process_dim
seq_times, seq_types, seq_lengths, _,seq_intensities = sahp.process_loaded_sequences(loaded_hawkes_data, process_dim)

total_sample_size = seq_times.size(0)
print("Total sample size: {}".format(total_sample_size))

train_ratio = 0.8
train_size = int(train_ratio * total_sample_size)
dev_ratio = 0.1
dev_size = int(dev_ratio * total_sample_size)
test_size = int(dev_ratio * total_sample_size)

print("Train sample size: {:}/{:}".format(train_size, total_sample_size))
print("Dev sample size: {:}/{:}".format(dev_size, total_sample_size))


# Define training data
train_times_tensor = seq_times[:train_size]
train_seq_types = seq_types[:train_size]
train_seq_lengths = seq_lengths[:train_size]
print("No. of event tokens in training subset:", train_seq_lengths.sum())

# Define development data
dev_times_tensor = seq_times[train_size:train_size + dev_size]  # train_size+dev_size
dev_seq_types = seq_types[train_size:train_size + dev_size]
dev_seq_lengths = seq_lengths[train_size:train_size + dev_size]
print("No. of event tokens in development subset:", dev_seq_lengths.sum())

test_times_tensor = seq_times[-dev_size:]
test_seq_types = seq_types[-dev_size:]
test_seq_lengths = seq_lengths[-dev_size:]
# test_seq_intensities = seq_intensities[-dev_size:]

print("No. of event tokens in test subset:", test_seq_lengths.sum())


train_seq_lengths, reorder_indices_train = train_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
train_times_tensor = train_times_tensor[reorder_indices_train]
train_seq_types = train_seq_types[reorder_indices_train]
#
dev_seq_lengths, reorder_indices_dev = dev_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
dev_times_tensor = dev_times_tensor[reorder_indices_dev]
dev_seq_types = dev_seq_types[reorder_indices_dev]

test_seq_lengths, reorder_indices_test = test_seq_lengths.sort(descending=True)
# # Reorder by descending sequence length
test_times_tensor = test_times_tensor[reorder_indices_test]
test_seq_types = test_seq_types[reorder_indices_test]
# test_seq_intensities = test_seq_intensities[reorder_indices_test]

device = 'cpu'
max_sequence_length = max(train_seq_lengths[0], dev_seq_lengths[0], test_seq_lengths[0])
print('max_sequence_length: {}'.format(max_sequence_length))
model = sahp.make_model(nLayers=4, d_model=16, atten_heads=8,
                   dropout=0.1, process_dim=2, device=device, pe='add',
                   max_sequence_length=max_sequence_length + 1).to(device)


model_dict =torch.load('trained_models/sahp/hawkes_2d_debug_model.pt',map_location=torch.device('cpu'))
model.load_state_dict(model_dict)

Total sample size: 4000
Train sample size: 3200/4000
Dev sample size: 400/4000
No. of event tokens in training subset: tensor(564977)
No. of event tokens in development subset: tensor(71399)
No. of event tokens in test subset: tensor(71543)
max_sequence_length: 399


<All keys matched successfully>

In [19]:
test_rmse, test_accuracy, results = \
    sahp.prediction_evaluation(device, model, train_seq_lengths, train_times_tensor, train_seq_types, train_size, tmax)

test_rmse, test_accuracy, results = \
    sahp.prediction_evaluation(device, model, test_seq_lengths, test_times_tensor, test_seq_types, test_size, tmax)

test_rmse, test_accuracy, results = \
    sahp.prediction_evaluation(device, model, dev_seq_lengths, dev_times_tensor, dev_seq_types, test_size, tmax)




100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:08<00:00, 46.26it/s]


rmse 2.148583979381555
Type prediction score: 0.5625


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:08<00:00, 45.66it/s]


rmse 1.771393055438939
Type prediction score: 0.5775


100%|██████████████████████████████████████████████████████████████████████████████| 3200/3200 [01:14<00:00, 43.00it/s]


rmse 1.9609460716516156
Type prediction score: 0.5675


### All Event Predictions

In [38]:
def rmse_for_all_events(test_times_tensor,test_seq_types,test_seq_lengths,t_max =200,device = 'cpu',n_samples=100):
    seq_types = test_seq_types
    seq_times = test_times_tensor
    seq_lengths = test_seq_lengths
    test_size = seq_times.shape[0]

    pad=model.process_dim
    hmax = tmax
    device ='cpu'

    all_event_errors = []

    for index in range(test_size):
        print('Sequence: {}'.format(index))
        times, types, lengths =seq_times[index], seq_types[index], seq_lengths[index]
        length = lengths.item()  # exclude the first added event

        all_event_errors+= predict_for_all_events(times, types,length,n_samples =n_samples)
        
    RMSE = np.sqrt(np.array(all_event_errors).mean())
    return RMSE

In [37]:
rmse_for_all_events(test_times_tensor,test_seq_types,test_seq_lengths,t_max =200,device = 'cpu',n_samples=500)

0.9530211731026779

In [34]:
def predict_for_all_events (seq_times, seq_types,length,n_samples = 1000):

    all_seq_errors =[]
    for i in range(length):

        event_index = i


        ## remove the first added event
        dt_seq = seq_times[1:] - seq_times[:-1]

        dt_seq = seq_times[1:] - seq_times[:-1]
        last_t = seq_times[length - 1 -event_index]
        next_t = seq_times[length-event_index]

        dt_seq_valid = dt_seq[:length]  # exclude the last timestamp
        dt_seq_used = dt_seq_valid[:length - 1-event_index]  # exclude the last timestamp
        next_dt = dt_seq_valid[length - 1-event_index]

        seq_types_valid = seq_types[1:length + 1-event_index]  # include the first added event
        last_type = seq_types[length - 1-event_index]
        next_type = seq_types[length-event_index]

        seq_types_used = seq_types_valid[:-1]
        seq_types_valid_masked = sahp.MaskBatch(seq_types_used[None, :], pad, device)
        seq_types_used_mask = seq_types_valid_masked.src_mask


        with torch.no_grad():
            model.forward(dt_seq_used, seq_types_used, seq_types_used_mask)

            if model.omega.shape[1] == 0:  # only one element
                estimate_dt, next_dt, error_dt, next_type, estimate_type = 0, 0, 0, 0, 0

            elif model.omega.shape[1] == 1:  # only one element
                converge_point = torch.squeeze(model.converge_point)[None, :]
                start_point = torch.squeeze(model.start_point)[None, :]
                omega = torch.squeeze(model.omega)[None, :]
            else:
                converge_point = torch.squeeze(model.converge_point)[-1, :]
                start_point = torch.squeeze(model.start_point)[-1, :]
                omega = torch.squeeze(model.omega)[-1, :]


            dt_vals = torch.linspace(0, hmax, n_samples + 1).to(device)
            h_t_vals = model.state_decay(converge_point,
                                        start_point,
                                        omega,
                                        dt_vals[:, None])


            a = sahp.predict_from_hidden(model, h_t_vals, dt_vals, next_dt, next_type,
                                               False, hmax, n_samples, False)
            all_seq_errors.append(a[2].item())
            
    return all_seq_errors

In [8]:
all_seq_errors

[1.5741888284683228,
 0.00020053365733474493,
 0.12581388652324677,
 0.4899369776248932,
 0.0001597424561623484,
 0.5166848301887512,
 0.051582079380750656,
 2.0383007526397705,
 0.16696572303771973,
 0.07290241867303848,
 0.028628166764974594,
 0.22603550553321838,
 0.05024284869432449,
 0.5963874459266663,
 0.007407496217638254,
 0.34040167927742004,
 0.4387223720550537,
 0.006443965248763561,
 0.012787806801497936,
 0.2756159007549286,
 0.0688042864203453,
 0.023869285359978676,
 0.11537044495344162,
 1.8956339359283447,
 0.12346295267343521,
 1.4260636227447776e-08,
 0.06775710731744766,
 0.09133005887269974,
 1.5678402633056976e-05,
 0.2439853698015213,
 0.0004917021724395454,
 0.33954986929893494,
 0.1624707132577896,
 0.008066406473517418,
 0.058877184987068176,
 0.0206342414021492,
 0.04852096363902092,
 3.1804120226297528e-06,
 0.07999629527330399,
 0.0029583480209112167,
 0.025910191237926483,
 0.0855279341340065,
 0.3716937005519867,
 0.05206200107932091,
 0.7400801181793213

In [62]:
batch_onehot = sahp.one_hot_embedding(test_seq_types, model.input_size)

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [61]:
batch_onehot[:,:,:].shape

torch.Size([400, 92, 2])

In [58]:
test_times_tensor[0]

tensor([  0.0000,   4.6665,   6.0890,  11.9530,  12.4728,  12.5900,  20.7698,
         21.7239,  21.8642,  22.5975,  26.8415,  27.9285,  28.7177,  30.1560,
         31.3296,  32.4544,  33.1469,  33.7802,  34.0427,  40.8920,  51.5232,
         53.5542,  54.1573,  56.5474,  57.7922,  62.8637,  73.4870,  75.0916,
         75.5580,  78.6898,  84.5311,  86.0235,  88.0580,  89.0156,  90.7996,
         90.8185,  92.8762,  99.6050, 100.1194, 100.8769, 101.1022, 102.2296,
        102.2321, 104.2340, 104.3581, 113.2720, 117.8082, 117.9675, 118.7762,
        119.9810, 122.4985, 122.5790, 123.4858, 124.5513, 126.1128, 127.1961,
        127.6598, 130.5543, 136.2862, 140.4792, 140.4970, 141.4220, 142.2272,
        143.1583, 143.8658, 145.5730, 151.0353, 151.5467, 151.7551, 153.1299,
        153.7074, 153.7725, 153.9533, 159.0476, 161.3204, 169.2330, 169.2663,
        170.7163, 180.0621, 180.1047, 186.5563, 195.6226, 196.5975, 198.4887,
        199.9994, 199.9994, 199.9994, 199.9994, 199.9994, 199.99